In [ ]:
## Workout Analytics -01, 02-
#### 01. Aquisition with Crawling
#### 02. Transformation included Cleansing, Denoising and Normalization

In [ ]:
import sys
import ssl
from datetime import datetime
from urllib.request import Request, urlopen
from functools import reduce
from itertools import count


resfetch = []
try:
    ssl._create_default_https_context = ssl._create_unverified_context

    for idx in count(start=1):
        l = len(resfetch)
        resfetch[l:] = list(filter(
            lambda s: s.startswith('#'),
            reduce(
                lambda param, func: func(param),
                [lambda raw: [] if raw is None else raw.splitlines(), lambda strs: [f(s) for f in [str.strip] for s in strs]],
                urlopen(Request('https://raw.githubusercontent.com/kickscar/HELLCHANG-PRACTICES/main/datasets/Week0%02d.md' % idx)).read().decode('utf-8', errors='replace')
            )
        ))

        if len(resfetch) == l:
            break
except Exception as err:
    (lambda e: print(f'{e} : {datetime.now()}', file=sys.stderr))(err)

print(resfetch)

In [ ]:
from pathlib import Path
import os
import pickle


# Load Exercise Dictionary
dictfile = os.path.join(Path(os.getcwd()).parent, 'datasets', 'exercise-dict.pkl')
with open(dictfile, 'rb') as file:
    exercisedict = pickle.load(file)

print(f'Loding Exercise Dictionary Pickle[{dictfile}]')

In [ ]:
import re
from datetime import datetime

# Tranformation
restransform = []
exs_dict = exercisedict['exercises']
eqptdials_dict = exercisedict['equipment-dialects']

for res in resfetch:
    if res.startswith('####'):
        # 1. split exercise name & record
        exname, exrecord = (s.strip() for s in res.replace('####', '').strip().split(':'))

        # 2. normalize exercise name with equipment
        exname = re.sub(r'[\d+\\.]', '', exname).strip()
        pttrmatch = re.compile(r'([a-zA-Z\s\',]+)\s*\[([a-zA-Z\s\',]+)\]').match(exname)
        exname, equipment = (exname, None) if pttrmatch is None else pttrmatch.groups()

        # 2-1. name
        exname = exname.strip().title()
        for exkey in exs_dict:
            exdials = exs_dict[exkey]['dialects']
            if exname in exdials:
                exname = exkey
                break

        # 2-2. equipment
        if equipment:
            equipment = equipment.strip().title()
            for eqptdials_key in eqptdials_dict:
                if equipment in eqptdials_dict[eqptdials_key]:
                    equipment = eqptdials_key
                    break

            exname = f'{exname}[{equipment}]'

        restransform[len(restransform):] = [(date, exname, *re.split(r'\s+', exrecord.strip()))]
    elif res.startswith('##'):
        date = datetime.strptime(res.replace('##', '').strip(), '%Y/%m/%d')

print(restransform)